In [9]:
import numpy as np
import sympy as sp
import timeit


x = sp.symbols('x') #declaration de la variable symbolique x
f_sym = 1 + x**2 #definition de la fonction symbolique


In [10]:
f = sp.lambdify(x, f_sym, 'numpy') #conversion de la fonction symbolique en fonction numerique

In [11]:
def methode_rectangle(a, b, m):
    h = (b -a)/ m
    x = np.linspace(a, b-h,m)
    return h * np.sum(f(x))

In [12]:
def methode_trapeze(a, b, m):
    h = (b - a)/ m
    x = np.linspace(a, b, m+1)
    y= f(x)
    return h/2 * (y[0] + 2 * np.sum(y[1:-1]) +y[-1])

In [13]:
def methode_simpson(a, b, m):
    if m % 2 != 0:
        raise ValueError("m doit être pair pour la méthode de Simpson.")
    h = (b - a) / m
    x = np.linspace(a, b, m + 1)
    y = f(x)
    somme_impairs = np.sum(y[1:-1:2])
    somme_pairs = np.sum(y[2:-1:2])
    return (h / 3) * (y[0] + 4 * somme_impairs + 2 * somme_pairs + y[-1])


In [14]:

def integrate_spline_quadratique(x_nodes, y_nodes, z0):
    
    n = len(x_nodes)
    z = np.zeros(n)
    z[0] = z0

    # 1Calcul des pentes z_i
    for i in range(1, n):
        dx = x_nodes[i] - x_nodes[i-1]
        dy = y_nodes[i] - y_nodes[i-1]
        z[i] = 2 * (dy/dx) - z[i-1]

    # 2Intégration analytique sur chaque morceau
    I_total = 0.0
    for i in range(n - 1):
        dx = x_nodes[i+1] - x_nodes[i]
        a = 0.5 * (z[i+1] - z[i]) / dx
        I_i = a*(dx**3)/3 + z[i]*(dx**2)/2 + y_nodes[i]*dx
        I_total += I_i

    return I_total


In [15]:
def integrate_lagrange(x_nodes, y_nodes, a=None, b=None):
   
    x_nodes = np.asarray(x_nodes)
    y_nodes = np.asarray(y_nodes)
    n = len(x_nodes)
    if a is None: a = x_nodes[0]
    if b is None: b = x_nodes[-1]

    P_total = np.poly1d([0.0]) #polynôme nul
    for i in range(n):
        Li = np.poly1d([1.0])#polynôme unité    
        for j in range(n):
            if i != j:
                Li *= np.poly1d([1.0, -x_nodes[j]]) / (x_nodes[i] - x_nodes[j]) #
        P_total += y_nodes[i] * Li

    P_int = np.polyint(P_total) #polynôme intégrale de P_total
    return float(np.polyval(P_int, b) - np.polyval(P_int, a)) #intégrale définie de a à b


In [16]:


# Intervalle et subdivisions
a, b = 0, 2
m = 4
x_nodes = np.linspace(a, b, m+1)
y_nodes = np.array([float(f_sym.subs(x, xi)) for xi in x_nodes])
z0 = float(sp.diff(f_sym, x).subs(x, x_nodes[0]))

I_exact = float(sp.integrate(f_sym, (x, a, b))) #intégrale exacte
print(f"Intégrale exacte = {I_exact:.12f}\n")

# -----------------------------
# Liste des méthodes à tester
# -----------------------------
# Chaque entrée = (nom_méthode, fonction, arguments)
methodes = [
    ("Rectangle", methode_rectangle, (a, b, m)),
    ("Trapèze", methode_trapeze, (a, b, m)),
    ("Simpson", methode_simpson, (a, b, m)),
    ("Spline quadratique", integrate_spline_quadratique, (x_nodes, y_nodes, z0)),
    ("Lagrange", integrate_lagrange, (x_nodes, y_nodes, x_nodes[0], x_nodes[-1]))
]

# -----------------------------
# 5Calcul et mesure des temps
# -----------------------------
resultats = []
for nom, fonction, args in methodes:
    start = timeit.timeit()
    valeur = fonction(*args)
    end = timeit.timeit()
    temps = end - start
    resultats.append((nom, valeur, temps))

# -----------------------------
# 6Tri par temps croissant
# -----------------------------
resultats.sort(key=lambda x: x[2])  # tri selon le temps d'exécution

# -----------------------------
# 7Affichage des résultats
# -----------------------------
print(f"{'Méthode':25s} {'Valeur':>20s}  {'Temps (s)':>12s}")
print("-"*80)
for nom, valeur, temps in resultats:
    erreur = abs(valeur - I_exact)
    print(f"{nom:25s} {valeur:20.12f}  {temps:12.6f}")

Intégrale exacte = 4.666666666667



Méthode                                 Valeur     Temps (s)
--------------------------------------------------------------------------------
Simpson                         4.666666666667     -0.063447
Rectangle                       3.750000000000     -0.063330
Trapèze                         4.750000000000     -0.040325
Spline quadratique              4.666666666667     -0.015823
Lagrange                        4.666666666667     -0.002118
